In [1]:
library(SnapATAC)
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(parallel)
library(foreach)

Loading required package: Matrix

Loading required package: rhdf5

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects ar

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240610/sc-E2G/test/results/K562_IGVF_filter/K562/Kendall/Pairs.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240610/sc-E2G/test/results/K562_IGVF_filter/K562/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_IGVF/1.prepare_data/2.Filter_out_low_quality_cells.240614/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_IGVF/3.Genome_wide_prediction/SnapATAC/SnapATAC.5Mb.240620/"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G.filter = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ATAC matrix

In [5]:
matrix.atac_count = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac_count)
rm(matrix.atac_count)

Import RNA matrix

In [6]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna_count = matrix.rna_count[rowSums(matrix.rna_count) > 0,]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

In [7]:
pairs.E2G.filter2 = pairs.E2G.filter[pairs.E2G.filter$TargetGene %in% rownames(matrix.rna) &
                                     pairs.E2G.filter$PeakName %in% rownames(matrix.atac)]

Run SnapATAC prediction

In [8]:
PredictionSnapATAC = function(pairs.E2G,
                              data.RNA,
                              data.ATAC,
                              n.core = 16){
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (index.tmp = 1:length(pairs.E2G),
             .combine = 'c',
             .packages = c("Matrix",
                           "GenomicRanges")) %dopar% {

                             pairs.E2G.tmp = pairs.E2G[index.tmp]
                             
                             model.tmp = summary(stats::glm(data.ATAC[pairs.E2G.tmp$PeakName,] ~ data.RNA[pairs.E2G.tmp$TargetGene,], 
                                                            family = binomial(link = "logit")))[["coefficients"]]
                             mcols(pairs.E2G.tmp)[,c("beta","sd","z","p")] = 
                               t(data.frame(model.tmp[2,]))
                             
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [9]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [10]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G.filter2)))
chr.run = chr.run[!chr.run %in% chr.done]

In [11]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr.res = pairs.E2G.filter2[seqnames(pairs.E2G.filter2) == chr.tmp]
  mcols(pairs.E2G.chr.res)[,c("beta","sd","z","p")] = NA
  pairs.E2G.chr.res = PredictionSnapATAC(pairs.E2G.chr.res,
                                         matrix.rna[rownames(matrix.rna) %in% pairs.E2G.chr.res$TargetGene,],
                                         matrix.atac[rownames(matrix.atac) %in% pairs.E2G.chr.res$PeakName,],
                                         n.cores)
  saveRDS(pairs.E2G.chr.res,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.res)
}

[1] "chr1"
[1] "chr10"
[1] "chr11"
[1] "chr12"
[1] "chr13"
[1] "chr14"
[1] "chr15"
[1] "chr16"
[1] "chr17"
[1] "chr18"
[1] "chr19"
[1] "chr2"
[1] "chr20"
[1] "chr21"
[1] "chr22"
[1] "chr3"
[1] "chr4"
[1] "chr5"
[1] "chr6"
[1] "chr7"
[1] "chr8"
[1] "chr9"
[1] "chrX"


Merge results

In [12]:
chr.merge = as.character(unique(seqnames(pairs.E2G.filter2)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [13]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 10023262 ranges and 7 metadata columns:
       seqnames            ranges strand |  TargetGene               PeakName
          <Rle>         <IRanges>  <Rle> | <character>            <character>
  chr1     chr1 10002642-10003186      * |     AADACL4 chr1-10002642-10003186
  chr1     chr1 10002642-10003186      * |       ACOT7 chr1-10002642-10003186
  chr1     chr1 10002642-10003186      * |      AGTRAP chr1-10002642-10003186
  chr1     chr1 10002642-10003186      * |    C1orf167 chr1-10002642-10003186
  chr1     chr1 10002642-10003186      * |      CAMTA1 chr1-10002642-10003186
   ...      ...               ...    ... .         ...                    ...
  chrX     chrX   9995829-9996183      * |       TBL1X   chrX-9995829-9996183
  chrX     chrX   9995829-9996183      * |      TCEANC   chrX-9995829-9996183
  chrX     chrX   9995829-9996183      * |      TMSB4X   chrX-9995829-9996183
  chrX     chrX   9995829-9996183      * |     TRAPPC2   chrX-9995829-9996183
  ch

In [14]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta",
                         "sd",
                         "z",
                         "p")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,sd,z,p
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10002642,10003186,AADACL4,K562,2.074901229,1.3383941,1.550291665,0.12107153
chr1,10002642,10003186,ACOT7,K562,-0.430184395,0.6426665,-0.669374209,0.50325680
chr1,10002642,10003186,AGTRAP,K562,0.463401855,0.6042724,0.766875702,0.44315542
chr1,10002642,10003186,C1orf167,K562,1.200180823,0.5437622,2.207179733,0.02730151
chr1,10002642,10003186,CAMTA1,K562,0.292543208,0.2436478,1.200680618,0.22987512
chr1,10002642,10003186,CASZ1,K562,-30.330467042,1684.8112444,-0.018002294,0.98563702
chr1,10002642,10003186,CHD5,K562,-33.146652257,2525.4147783,-0.013125231,0.98952788
chr1,10002642,10003186,CLCN6,K562,0.274636465,0.3302594,0.831577985,0.40564719
chr1,10002642,10003186,CLSTN1,K562,-0.044401159,0.3739474,-0.118736389,0.90548421


In [15]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] foreach_1.5.2        GenomicRanges_1.54.1 GenomeInfoDb_1.38.1 
 [4] IRanges_2.36.0       S4Vectors_0.40.2     BiocGene